### Startup

In [ ]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
from tqdm import tqdm
import random, json, copy
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

In [ ]:
sensitivity_path = os.getcwd() + "/classification/sensitivity/"
print(sensitivity_path)

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [ ]:
cfg = breaching.get_config(overrides=["case=6_large_batch_cifar"])

device = torch.device(f'cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [ ]:
cfg.case.data.partition="unique-class"
# cfg.case.data.partition="balanced" # 100 unique CIFAR-100 images
cfg.case.user.num_data_points = 1
cfg.case.user.user_idx = 1
cfg.case.model='CNN_FedAvg'
# convnet

cfg.case.user.provide_labels = False
cfg.attack.label_strategy = "yin" # also works here, as labels are unique

# Total variation regularization needs to be smaller on CIFAR-10:
cfg.attack.regularization.total_variation.scale = 5e-4

# Reduce max iteration to save time
# cfg.attack.optim.max_iterations = 100


### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [ ]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
model_path = f"./classification/model-{cfg.case.model}-cifar.pt"
# torch.save(model.state_dict(), model_path)
model.load_state_dict(torch.load(model_path))
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

In [ ]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

In [ ]:
user.plot(true_user_data)

**Network Info**

In [ ]:
num_layer = len(shared_data['gradients'])
print(f"Number of Layers: {num_layer}")

In [ ]:
num_param = 0
num_param_list = []
for layer in shared_data['gradients']:
    num_param += layer.numel()
    num_param_list.append(layer.numel())
print(f"Number of Parameters: {num_param}")

In [ ]:
num_param_list

In [ ]:
from sewar.full_ref import msssim, uqi, vifp

def get_sim_metrics(true_data, reconstructed_data):
    def scale_tensor(d):
        min_val, max_val = d.amin(dim=[2, 3], keepdim=True), d.amax(dim=[2, 3], keepdim=True)
        # print(f'min_val: {min_val} | max_val: {max_val}')
        return (d - min_val) / (max_val - min_val)

    true_data_scaled = scale_tensor(true_data["data"]).squeeze().permute(1, 2, 0).cpu().numpy()
    reconstructed_data_scaled = scale_tensor(reconstructed_data["data"]).squeeze().permute(1, 2, 0).cpu().numpy()

    metrics = {
        "msssim": msssim(true_data_scaled, reconstructed_data_scaled, MAX=1.0),
        "uqi": uqi(true_data_scaled, reconstructed_data_scaled),
        "vifp": vifp(true_data_scaled, reconstructed_data_scaled)
    }
    return metrics

## No Encryption

In [ ]:
results_no_dict = {"mse": [], "msssim": [], "uqi": [], "vifp": []}
for _ in tqdm(range(10)):
    reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], {}, dryrun=cfg.dryrun)
    metrics = breaching.analysis.report(reconstructed_user_data, 
                                        true_user_data, [server_payload], 
                                        server.model, order_batch=True, compute_full_iip=False, 
                                        cfg_case=cfg.case, setup=setup)
    sim_score = get_sim_metrics(true_user_data, reconstructed_user_data)
    results_dict_temp = {
        "mse": metrics["mse"],
        "msssim": sim_score["msssim"],
        "uqi": sim_score["uqi"],
        "vifp": sim_score["vifp"]
    }
    for k, v in results_dict_temp.items():
        results_no_dict[k].append(v)
        print(f"{k}: {v}")
    user.plot(reconstructed_user_data)
torch.save(results_no_dict, f"./classification/results/{cfg.case.user.user_idx}/{cfg.case.model}_cifar_no_protection.pt")

## Selective Encryption

In [ ]:
def flat_tensor_list(grads):
    flat_grads = torch.tensor([], device=grads[0].device)
    for i in grads:
        flat_grads = torch.concat((flat_grads, i.view(-1)), dim = 0)
    return flat_grads

def encrypt_by_index(grad_list: list, idx_list: float, if_plot=False) -> list:
    """
    @grad_list: gradient list to be encrypted
    @idx_list: list of encrypted parameter indices
    @if_plot: whether to plot the encrypted ratio of each layer
    """
    grad_flat = flat_tensor_list(grad_list)

    mask = torch.zeros(grad_flat.shape).to(device=grad_flat.device) == 0
    mask[idx_list] = False

    # Apply the encryption mask
    grad_flat = grad_flat * mask + torch.rand(grad_flat.shape).to(device=grad_flat.device) * ~mask

    # Reshape back
    new_grad_list = []
    start_idx = 0
    ratio_enc_per_layer = []
    for layer_grad in tqdm(grad_list):
        layer_param_num = layer_grad.numel()
        grad_slice = grad_flat[start_idx: start_idx + layer_param_num]
        new_grad_list.append(grad_slice.reshape(layer_grad.shape))

        # Record the number of encrypted parameters of each layer
        mask_slice = mask[start_idx: start_idx + layer_param_num]
        ratio_param_enc_layer = (layer_param_num - mask_slice.sum()) / layer_param_num
        ratio_enc_per_layer.append(ratio_param_enc_layer.item())

        start_idx += layer_param_num

    if if_plot:
        plt.bar(np.arange(1, len(ratio_enc_per_layer)+1), ratio_enc_per_layer)
        plt.xlabel("Layer Index")
        plt.ylabel("Encryption Ratio")
        plt.show()
    return new_grad_list

In [ ]:
def encrypt_selective(sens_list, ratio_enc, num_repeat=1):
    num_enc = (int) (ratio_enc * num_param)
    encrypt_list = torch.topk(flat_tensor_list(sens_list).abs(), num_enc, largest=True).indices

    results_dict = {"mse": [], "msssim": [], "uqi": [], "vifp": []}
    for _ in tqdm(range(num_repeat), desc=f"p={ratio_enc}"):
        new_gradient = copy.deepcopy(shared_data['gradients'])
        new_gradient = encrypt_by_index(new_gradient, encrypt_list)

        gradient_with_he = list(new_gradient)
        new_shared_data = dict()
        for k in shared_data.keys():
            if k == "gradients":
                new_shared_data[k] = gradient_with_he
            else:
                new_shared_data[k] = shared_data[k]

        # print(new_shared_data['gradients'])

        new_reconstructed_user_data, stats = attacker.reconstruct([server_payload],
                                                                [new_shared_data],
                                                                server.secrets,
                                                                dryrun=cfg.dryrun)
        metrics = breaching.analysis.report(new_reconstructed_user_data,
                                            true_user_data,
                                            [server_payload],
                                            server.model,
                                            order_batch=True,
                                            compute_full_iip=False,
                                            cfg_case=cfg.case,
                                            setup=setup)
        sim_score = get_sim_metrics(true_user_data, new_reconstructed_user_data)
        results_dict_temp = {
            "mse": metrics["mse"],
            "msssim": sim_score["msssim"],
            "uqi": sim_score["uqi"],
            "vifp": sim_score["vifp"]
        }
        print(f"Encrypted the {ratio_enc * 100:.2f}% parameters")
        for k, v in results_dict_temp.items():
            results_dict[k].append(v)
            print(f"{k}: {v}")
        user.plot(new_reconstructed_user_data)
    return results_dict

In [ ]:
# Load sensitivity
sens_path = sensitivity_path + f"{cfg.case.model}_cifar_mean_sens.pt"
sens_list = torch.load(sens_path)

for ratio in [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    results_select_dict = encrypt_selective(sens_list, 
                                            ratio_enc=ratio,
                                            num_repeat=10)
    torch.save(results_select_dict,
               f"./classification/results/{cfg.case.user.user_idx}/{cfg.case.model}_cifar_selective_{ratio}.pt")

## Random Encryption

In [ ]:
# below are for for random encryption by parameters (modified):

def encrypt_random(ratio_enc, num_repeat=1):
    # Randomly pick a proportion to encrypt
    protected_params = random.sample(range(num_param), (int) (ratio_enc * num_param))

    results_dict = {"mse": [], "msssim": [], "uqi": [], "vifp": []}
    for _ in tqdm(range(num_repeat), desc=f"p={ratio_enc}"):
        new_gradient = copy.deepcopy(shared_data['gradients'])
        new_gradient = encrypt_by_index(new_gradient, protected_params)

        gradient_with_he = list(new_gradient)
        new_shared_data = dict()
        for k in shared_data.keys():
            if k == "gradients":
                new_shared_data[k] = gradient_with_he
            else:
                new_shared_data[k] = shared_data[k]

        # print(new_shared_data['gradients'])

        new_reconstructed_user_data, stats = attacker.reconstruct([server_payload],
                                                                [new_shared_data],
                                                                server.secrets,
                                                                dryrun=cfg.dryrun)
        metrics = breaching.analysis.report(new_reconstructed_user_data,
                                            true_user_data,
                                            [server_payload],
                                            server.model,
                                            order_batch=True,
                                            compute_full_iip=False,
                                            cfg_case=cfg.case,
                                            setup=setup)
        sim_score = get_sim_metrics(true_user_data, new_reconstructed_user_data)
        results_dict_temp = {
            "mse": metrics["mse"],
            "msssim": sim_score["msssim"],
            "uqi": sim_score["uqi"],
            "vifp": sim_score["vifp"]
        }
        print(f"Randomly encrypted {ratio_enc * 100}% parameters")
        for k, v in results_dict_temp.items():
            results_dict[k].append(v)
            print(f"{k}: {v}")
        user.plot(new_reconstructed_user_data)
    return results_dict

In [ ]:
for ratio in [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    results_random_list = []
    for i in tqdm(range(1), desc="Testing random init..."):
        results_dict = encrypt_random(ratio_enc=ratio, num_repeat=10)
        results_random_list.append(results_dict)
    torch.save(results_random_list,
               f"./classification/results/{cfg.case.model}_cifar_random_{ratio}.pt")